# Finding the hip joint centre in global coordinate system

> Alex Woodall, Thor Besier <br>
> Auckland Bioengineering Institute <br>
> University of Auckland, New Zealand

This notebook will open up a __static__ trial.trc file and calculate the hip joint centres within the global coordinate system.

Start by importing the libraries needed to run the task.

In [2]:
# Import the necessary libraries
import numpy as np

import sys
sys.path.insert(1, r'./../Functions')  # add to path

from read_trc import read_trc
from segment_orientation_V1V3 import segment_orientation_V1V3

## Reading in a TRC file

We can read in the desired trc file, containing the coordinate data for each of the markers during a motion capture recording session.

In [3]:
mkr_data, _ = read_trc()

nframes, ncolumns = np.shape(mkr_data["Data"]["RawData"])

You have selected Cancel. Please select a TRC file.
Leaving readTRC


TypeError: cannot unpack non-iterable NoneType object

## Choosing the markers

To calculate the hip joint centre, we need the markers on the hip.

The hip markers also allow us to define a marker for the sacrum (half way between the LPsis and RPsis) and the pelvis origin (half way between the LAsis and RAsis). If there is a sacrum marker in the marker set, feel free to use that instead of calculating it.

In [3]:
LASI = mkr_data["Data"]["Markers"]["LAsis"]["All"]
RASI = mkr_data["Data"]["Markers"]["RAsis"]["All"]
LPSI = mkr_data["Data"]["Markers"]["LPsis"]["All"]
RPSI = mkr_data["Data"]["Markers"]["RPsis"]["All"]

In [4]:
# Define sacrum marker
SACR = (LPSI + RPSI)/2

# Define the pelvis origin
origin_pelvis = (LASI + RASI)/2

## Calculating unit vectors

The functional method requires us to calculate the unit vectors of the pelvis segment relative to the global coordinate system. For more information on this, see the _segment_orientation_V1V3()_ function.

In [5]:
# Calculate unit vectors of pelvis segment relative to global
e1_pelvis, e2_pelvis, e3_pelvis = segment_orientation_V1V3(origin_pelvis - SACR, RASI - LASI)

## Global coordinate system

We can now project the hip joint centre back into the global coordinate system

In [6]:
# Initialise arrays for hip centres and define the left and right HJC locations (set to origin as default)
LHJC = []
RHJC = []
LHJC_location = np.zeros((3,1))
RHJC_location = np.zeros((3,1))

# Project hip joint centre back into global coordinate system
for i in range(nframes):
    rotation_matrix = np.array([e1_pelvis[i,:], e2_pelvis[i,:], e3_pelvis[i,:]])
    
    LHJC.append(np.matmul(np.linalg.inv(rotation_matrix), LHJC_location) + (origin_pelvis[i,:])[:,np.newaxis])
    RHJC.append(np.matmul(np.linalg.inv(rotation_matrix), RHJC_location) + (origin_pelvis[i,:])[:,np.newaxis])

# Convert to array
LHJC = np.array(LHJC)
RHJC = np.array(RHJC)

## Visualisation

Now you can export the LHJC and RHJC data and append it to the static.trc file... good luck!